# Métodos Mágicos: Dando "Superpoderes" a tus Clases

Los **métodos mágicos** (o *dunder methods*, por *Double Underscore*) son funciones especiales que se identifican por tener un doble guion bajo al principio y al final (ej. `__init__`, `__str__`).

Su "magia" es que **no los llamas directamente**. Python los llama por ti cuando usas una sintaxis u operación nativa del lenguaje sobre tus objetos. Por ejemplo, al hacer `print(mi_objeto)`, Python busca y ejecuta el método `__str__` de ese objeto.

## 1. Representación de Objetos: `__str__` y `__repr__`

Por defecto, imprimir un objeto muestra un mensaje poco útil. Con estos métodos, podemos definir cómo queremos que nuestros objetos se representen como texto.

* **`__str__(self)`:** Define una representación **legible e informal** para el usuario final. Es lo que usa `print()`.
* **`__repr__(self)`:** Define una representación **oficial y técnica** para el desarrollador, útil para depuración.

In [ ]:
class Persona:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

    # Salida para el usuario final
    def __str__(self):
        return f'Persona: {self.nombre}, {self.edad} años'

    # Salida para el desarrollador (debugging)
    def __repr__(self):
        return f'<Persona(nombre={self.nombre}, edad={self.edad})>'

# Creamos una instancia
p1 = Persona('Ana', 28)

# print() usa __str__
print(p1)

# repr() o simplemente ver el objeto en una celda de notebook usa __repr__
repr(p1)

## 2. Comparación de Objetos

Por defecto, Python no sabe cómo comparar dos objetos `Persona`. Necesitamos enseñarle usando los métodos mágicos de comparación.

* **`__eq__(self, otro)`:** Define el comportamiento del operador de igualdad `==`.
* **`__lt__(self, otro)`:** Define el comportamiento de "menor que" `<`.
* **`__le__(self, otro)`:** Define el comportamiento de "menor o igual que" `<=`.

In [ ]:
class Persona:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

    # Le decimos a Python que dos personas son iguales si su nombre y edad coinciden
    def __eq__(self, otro):
        print(f"Comparando {self.nombre} == {otro.nombre}")
        return self.nombre == otro.nombre and self.edad == otro.edad

    # Le decimos que una persona es "menor que" otra si su edad es menor
    def __lt__(self, otro):
        print(f"Comparando {self.edad} < {otro.edad}")
        return self.edad < otro.edad

# Creamos las instancias
p1 = Persona('Ana', 28)
p2 = Persona('Luis', 35)
p3 = Persona('Ana', 28)

# Ahora podemos comparar los objetos
print(f"¿p1 es igual a p2? {p1 == p2}")
print(f"¿p1 es igual a p3? {p1 == p3}")
print(f"¿p1 es menor que p2? {p1 < p2}")

## 3. Operaciones Aritméticas con Objetos

También podemos definir cómo se comportan nuestros objetos con operadores aritméticos como `+`.

* **`__add__(self, otro)`:** Define el comportamiento del operador de suma `+`.

In [ ]:
class Persona:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

    # Le decimos a Python que al "sumar" dos personas, lo que queremos es sumar sus edades
    def __add__(self, otro):
        print("Sumando las edades...")
        return self.edad + otro.edad

# Creamos las instancias
p1 = Persona('Ana', 28)
p2 = Persona('Luis', 35)

# Ahora podemos "sumar" los objetos
suma_edades = p1 + p2

print(f"La suma de las edades de Ana y Luis es: {suma_edades}")

# Métodos Mágicos Avanzados: `__new__`, `__init__` y `__call__`

En este notebook, exploraremos tres de los métodos mágicos más avanzados que nos dan un control total sobre el ciclo de vida y el comportamiento de nuestros objetos.

**La Analogía: Una Fábrica de Robots Multiplicadores**

Imagina que la clase `MultiplierFactory` es una **fábrica** que construye y programa **robots**. Cada robot tiene una sola misión: tomar un número y multiplicarlo por un "factor" secreto que se le asigna durante su construcción.

## 1. El Proceso de Construcción

Cuando ejecutas `multiplier = MultiplierFactory(5)`, ocurren dos pasos en la fábrica, en un orden muy específico.

### Paso A: `__new__` (El Ensamblaje del Chasis)
* **¿Qué es?** Es el **primer** método que se llama, incluso antes de `__init__`. Su trabajo es **construir el objeto**, la carcasa vacía del robot. Aún no tiene software ni personalidad.
* **¿Cómo funciona?** La línea `super().__new__(cls)` es el comando que le dice a Python: "usa tu maquinaria interna estándar para construir esta carcasa vacía".

### Paso B: `__init__` (La Programación)
* **¿Qué es?** Se llama **después** de que `__new__` ha creado el objeto. Su trabajo es **inicializar** o **configurar** ese objeto ya existente.
* **¿Cómo funciona?** Este es el ingeniero de software que instala el programa en el chasis. Toma el `factor` (5) y lo guarda en la memoria del robot (`self.factor = 5`).

## 2. El Proceso de Acción: `__call__`

Este método es el que le da al objeto su comportamiento principal después de haber sido creado.

### El Método `__call__` (El Botón de Activación)
* **¿Qué es?** Este método mágico le da al **objeto ya creado** (`multiplier`) la capacidad de ser **llamado como si fuera una función**.
* **¿Cómo funciona?** Es el **gran botón rojo** en el pecho del robot. Cuando "llamas" al objeto (`multiplier(10)`), estás presionando ese botón. El robot (el objeto `self`) toma el número que le das (`10`) y realiza la operación para la que fue programado (`10 * self.factor`).

In [12]:
class MultiplierFactory:

    # Se llama PRIMERO. Su trabajo es CONSTRUIR el objeto.
    def __new__(cls, factor: int):
        print(f"Paso 1 (__new__): Creando la instancia (el chasis del robot).")
        # Llama al constructor de la clase base para crear el objeto vacío.
        return super(MultiplierFactory, cls).__new__(cls)

    # Se llama SEGUNDO. Su trabajo es INICIALIZAR el objeto ya creado.
    def __init__(self, factor: int):
        print(f"Paso 2 (__init__): Inicializando y guardando el factor {factor}.")
        # Guarda el 'factor' como un atributo del objeto.
        self.factor = factor

    # Se llama cuando el OBJETO es tratado como una FUNCIÓN.
    def __call__(self, number: int) -> int:
        print(f"Paso 3 (__call__): 'Ejecutando' el objeto con el número {number}.")
        # Realiza la operación para la que fue programado.
        return number * self.factor

# --- Simulación ---
print("--- Creando el objeto 'multiplier' con factor 5 ---")
multiplier = MultiplierFactory(5)

print("\n--- 'Llamando' al objeto 'multiplier' con el número 10 ---")
result = multiplier(10)

print(f"\nEl resultado final es: {result}")

--- Creando el objeto 'multiplier' con factor 5 ---
Paso 1 (__new__): Creando la instancia (el chasis del robot).
Paso 2 (__init__): Inicializando y guardando el factor 5.

--- 'Llamando' al objeto 'multiplier' con el número 10 ---
Paso 3 (__call__): 'Ejecutando' el objeto con el número 10.

El resultado final es: 50
